In [ ]:
!pip install transformers==3.1.0

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification")
# classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.


In [ ]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics"]

classifier(sequence, candidate_labels)

{'labels': ['politics', 'economics', 'public health'],
 'scores': [0.972518801689148, 0.01458414364606142, 0.012897025793790817],
 'sequence': 'Who are you voting for in 2020?'}

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.


In [ ]:
sequence = "Who are you voting for in 2022 ?"
candidate_labels = ["politics", "public health", "economics", "elections"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['politics', 'elections', 'public health', 'economics'],
 'scores': [0.972069501876831,
  0.967610776424408,
  0.03248710557818413,
  0.0061644683592021465],
 'sequence': 'Who are you voting for in 2020?'}

Here's an example of sentiment classification:


In [ ]:
sequence = "I hated this movie."
candidate_labels = ["positive", "negative"]

classifier(sequence, candidate_labels)

{'labels': ['negative', 'positive'],
 'scores': [0.9916268587112427, 0.00837317667901516],
 'sequence': 'I hated this movie. The acting sucked.'}

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'labels': ['negative', 'positive'],
  'scores': [0.9916267991065979, 0.008373182266950607],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['negative', 'positive'],
  'scores': [0.8148515820503235, 0.1851484179496765],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)


In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'labels': ['negative', 'positive'],
  'scores': [0.9890093207359314, 0.010990672744810581],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['positive', 'negative'],
  'scores': [0.9581228494644165, 0.0418771356344223],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

By providing a more precise hypothesis template, we are able to see a more accurate classification of the second review.

> Note that sentiment classification is used here just as an illustrative example. The [Hugging Face Model Hub](https://huggingface.co/models?filter=text-classification) has a number of models trained specifically on sentiment tasks which can be used instead.


#### Update: Zero-shot classification in 100 languages

Interested in using the pipeline for languages other than English? We've trained a cross-lingual model on top of XLM RoBERTa which you can use by passing `model='joeddav/xlm-roberta-large-xnli'` when creating the pipeline:


In [ ]:
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

You can use it with any combination of languages. For example, let's classify a Russian sentence with English candidate labels:


In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "public health", "politics"]

classifier(sequence, candidate_labels)

{'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.9048484563827515, 0.05722189322113991, 0.03792969882488251],
 'sequence': 'За кого вы голосуете в 2020 году?'}

Now let's do the same but with the labels in French:


In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "santé publique", "politique"]

classifier(sequence, candidate_labels)

{'labels': ['politique', 'Europe', 'santé publique'],
 'scores': [0.9726154804229736, 0.017128489911556244, 0.010256024077534676],
 'sequence': 'За кого вы голосуете в 2020 году?'}

As we discussed in the last section, the default hypothesis template is the English, `This text is {}.`. If you are working strictly within one language, it may be worthwhile to translate this to the language you are working with:


In [ ]:
sequence = "¿A quién vas a votar en 2020?"
candidate_labels = ["Europa", "salud pública", "política"]
hypothesis_template = "Este ejemplo es {}."

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'labels': ['política', 'Europa', 'salud pública'],
 'scores': [0.9109585881233215, 0.05954807624220848, 0.029493311420083046],
 'sequence': '¿A quién vas a votar en 2020?'}